# Pasul 3 – Curățare, aliniere și combinare (EIA + WDI)

**Scop:** Curățarea datelor colectate din sursele EIA și World Development Indicators (WDI),
alinierea acestora pe aceeași unitate de analiză și combinarea într-un singur dataset final,
pregătit pentru analiza exploratorie și statistică.

## Unitatea de analiză

Unitatea de analiză utilizată în acest proiect este **țară × an**
(*countryRegionId × year*).  
Această alegere este justificată de natura anuală a indicatorilor EIA și WDI și permite
comparabilitatea directă între indicatorii energetici și cei socio-economici.

După alinierea surselor, dataset-ul final conține un panel echilibrat pentru
șase țări și perioada 2020–2023.

## Date de intrare

În această etapă sunt utilizate două seturi de date rezultate din Pasul 2:

- **Date EIA** (`01_eia_data_catalinaminciuna.csv`): indicatori energetici anuali
  (Primary Energy, Energy Intensity, CO₂ Emissions), în format long.
- **Date WDI** (`02_wdi_data_catalinaminciuna.csv`): indicatori socio-economici
  (PIB per capita, consum de energie, acces la electricitate, ponderea energiilor regenerabile),
  în format long.

Ambele seturi folosesc aceleași chei principale: `countryRegionId` și `year`.

In [5]:
import pandas as pd

# Citire corectă CSV (separator = virgulă)
eia = pd.read_csv("01_eia_data_catalinaminciuna.csv")
wdi = pd.read_csv("02_wdi_data_catalinaminciuna.csv")

print("EIA shape:", eia.shape)
print("WDI shape:", wdi.shape)

eia.head()

EIA shape: (216, 6)
WDI shape: (96, 4)


,countryRegionId,year,productId,value,unit,activityId
0,BGR,2020,44,9.417170,MTOE,1
1,BGR,2020,44,0.373704,QBTU,1
2,BGR,2020,44,394278.081589,TJ,1
3,BGR,2020,44,16.681647,MTOE,2
4,BGR,2020,44,0.661981,QBTU,2


## Verificări inițiale

Sunt verificate:
- tipurile de date (în special existența și formatul numeric al variabilei `year`);
- setul de țări și intervalul de ani disponibile în fiecare dataset;
- existența duplicatelor pe cheile principale (`countryRegionId`, `year`),
  respectiv (`countryRegionId`, `year`, `productId` pentru EIA).

In [6]:
print("EIA columns:", eia.columns.tolist())
print("WDI columns:", wdi.columns.tolist())

print("\nEIA dtypes:")
print(eia.dtypes)

print("\nWDI dtypes:")
print(wdi.dtypes)

EIA columns: ['countryRegionId', 'year', 'productId', 'value', 'unit', 'activityId']
WDI columns: ['countryRegionId', 'year', 'indicator', 'value']

EIA dtypes:
countryRegionId     object
year                 int64
productId            int64
value              float64
unit                object
activityId           int64
dtype: object

WDI dtypes:
countryRegionId     object
year                 int64
indicator           object
value              float64
dtype: object


In [7]:
print("EIA ani:", sorted(eia["year"].unique()))
print("WDI ani:", sorted(wdi["year"].unique()))

print("EIA țări:", sorted(eia["countryRegionId"].unique()))
print("WDI țări:", sorted(wdi["countryRegionId"].unique()))

EIA ani: [np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]
WDI ani: [np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]
EIA țări: ['BGR', 'CZE', 'HUN', 'POL', 'ROU', 'SVK']
WDI țări: ['BGR', 'CZE', 'HUN', 'POL', 'ROU', 'SVK']


In [8]:
eia_dups = eia.duplicated(subset=["countryRegionId","year","productId"]).sum()
print("Duplicate EIA (country, year, productId):", eia_dups)

# vedem ce unități/activity apar pentru fiecare productId
print("\nUnități pe productId:")
print(eia.groupby("productId")["unit"].value_counts())

print("\nActivityId pe productId:")
print(eia.groupby("productId")["activityId"].value_counts())

Duplicate EIA (country, year, productId): 144

Unități pe productId:
productId  unit     
44         MTOE         48
           QBTU         48
           TJ           48
47         MBTUPP       24
           TBTUUSDPP    24
4008       MMTCD        24
Name: count, dtype: int64

ActivityId pe productId:
productId  activityId
44         1             72
           2             72
47         33            24
           34            24
4008       8             24
Name: count, dtype: int64


## Curățarea datelor EIA

Datele EIA pot conține mai multe observații pentru aceeași combinație
(*țară, an, indicator*), deoarece același indicator este furnizat:
- în unități diferite (ex. MTOE, QBTU, TJ);
- în serii diferite, identificate prin `activityId`.

Pentru a obține un dataset coerent și utilizabil în analiză, a fost selectată
o **singură serie standard pentru fiecare indicator EIA**, prin alegerea unei
unități și a unui `activityId` reprezentativ.  
După această selecție, fiecare combinație (*țară × an × indicator*) este
reprezentată de o singură observație, eliminând duplicatele.

In [10]:
EIA_SERIES_RULES = {
    44: {"unit": "MTOE", "activityId": 1},   # Primary energy: păstrez seria în MTOE
    47: {"unit": "MBTUPP", "activityId": 33},# Energy intensity: aleg o singură serie/unitate
    4008: {"unit": "MMTCD", "activityId": 8} # CO2: seria standard
}

def select_eia_series(df):
    parts = []
    for pid, rules in EIA_SERIES_RULES.items():
        sub = df[df["productId"] == pid].copy()
        sub = sub[sub["unit"] == rules["unit"]]
        sub = sub[sub["activityId"] == rules["activityId"]]
        parts.append(sub)
    return pd.concat(parts, ignore_index=True)

eia_clean = select_eia_series(eia)

print("EIA clean shape:", eia_clean.shape)
print("Duplicate EIA clean:", eia_clean.duplicated(subset=["countryRegionId","year","productId"]).sum())
eia_clean.head(10)


EIA clean shape: (72, 6)
Duplicate EIA clean: 0


,countryRegionId,year,productId,value,unit,activityId
0,BGR,2020,44,9.417170,MTOE,1
1,BGR,2021,44,10.829147,MTOE,1
2,BGR,2022,44,11.916742,MTOE,1
3,BGR,2023,44,9.388583,MTOE,1
4,CZE,2020,44,20.962717,MTOE,1
5,CZE,2021,44,21.201138,MTOE,1
6,CZE,2022,44,22.481919,MTOE,1
7,CZE,2023,44,20.611831,MTOE,1
8,HUN,2020,44,8.685582,MTOE,1
9,HUN,2021,44,8.590009,MTOE,1


### Transformarea EIA în wide

In [11]:
eia_wide = (
    eia_clean
    .pivot_table(index=["countryRegionId","year"], columns="productId", values="value", aggfunc="first")
    .reset_index()
)

eia_wide = eia_wide.rename(columns={
    44: "eia_primary_energy_44",
    47: "eia_energy_intensity_47",
    4008: "eia_co2_emissions_4008"
})

print("EIA wide:", eia_wide.shape)
eia_wide.head()


EIA wide: (24, 5)


productId,countryRegionId,year,eia_primary_energy_44,eia_energy_intensity_47,eia_co2_emissions_4008
0,BGR,2020,9.417170,101.126811,35.177193
1,BGR,2021,10.829147,111.358190,40.654544
2,BGR,2022,11.916742,115.032870,44.326275
3,BGR,2023,9.388583,101.609508,33.465186
4,CZE,2020,20.962717,147.140623,88.581957


## Curățarea datelor WDI

Datele WDI au fost inițial normalizate în format long.  
În această etapă, ele au fost transformate în format wide, cu o coloană distinctă
pentru fiecare indicator socio-economic.

Pentru indicatorul *renewables_share* au fost identificate valori lipsă pentru
anumite țări și ani. Acestea au fost completate prin **interpolare liniară la nivel
de țară**, o metodă adecvată pentru serii temporale scurte și continue.


In [12]:
wdi_wide = (
    wdi
    .pivot_table(
        index=["countryRegionId", "year"],
        columns="indicator",
        values="value",
        aggfunc="first"
    )
    .reset_index()
)

print("WDI wide shape:", wdi_wide.shape)
wdi_wide.head()


WDI wide shape: (24, 6)


indicator,countryRegionId,year,electricity_access,energy_use_per_capita,gdp_per_capita,renewables_share
0,BGR,2020,99.7,2664.551530,10769.947570,21.1
1,BGR,2021,99.8,2918.916414,12972.217336,20.4
2,BGR,2022,100.0,2968.656634,13999.671920,NaN
3,BGR,2023,100.0,2604.191045,15883.861855,NaN
4,CZE,2020,100.0,3764.471612,23472.891545,17.0


In [13]:
print("Chei EIA:", eia_wide[["countryRegionId","year"]].shape)
print("Chei WDI:", wdi_wide[["countryRegionId","year"]].shape)

# verificăm dacă există combinații lipsă
missing_in_wdi = set(map(tuple, eia_wide[["countryRegionId","year"]].values)) - \
                 set(map(tuple, wdi_wide[["countryRegionId","year"]].values))

missing_in_eia = set(map(tuple, wdi_wide[["countryRegionId","year"]].values)) - \
                 set(map(tuple, eia_wide[["countryRegionId","year"]].values))

print("Chei EIA fără WDI:", missing_in_wdi)
print("Chei WDI fără EIA:", missing_in_eia)


Chei EIA: (24, 2)
Chei WDI: (24, 2)
Chei EIA fără WDI: set()
Chei WDI fără EIA: set()


## Combinarea seturilor de date EIA și WDI

După curățare și transformare, seturile de date EIA și WDI au fost combinate
printr-un join de tip *inner* pe cheile comune (`countryRegionId`, `year`).

Această abordare asigură păstrarea doar a observațiilor pentru care există
date complete în ambele surse, evitând introducerea valorilor lipsă artificiale.

In [14]:
final_data = pd.merge(
    eia_wide,
    wdi_wide,
    on=["countryRegionId", "year"],
    how="inner"
)

print("Dataset final:", final_data.shape)
final_data.head()


Dataset final: (24, 9)


,countryRegionId,year,eia_primary_energy_44,eia_energy_intensity_47,eia_co2_emissions_4008,electricity_access,energy_use_per_capita,gdp_per_capita,renewables_share
0,BGR,2020,9.417170,101.126811,35.177193,99.7,2664.551530,10769.947570,21.1
1,BGR,2021,10.829147,111.358190,40.654544,99.8,2918.916414,12972.217336,20.4
2,BGR,2022,11.916742,115.032870,44.326275,100.0,2968.656634,13999.671920,NaN
3,BGR,2023,9.388583,101.609508,33.465186,100.0,2604.191045,15883.861855,NaN
4,CZE,2020,20.962717,147.140623,88.581957,100.0,3764.471612,23472.891545,17.0


In [15]:
print("Valori lipsă pe coloană:\n", final_data.isna().sum())

dups = final_data.duplicated(subset=["countryRegionId","year"]).sum()
print("Duplicate finale (country, year):", dups)


Valori lipsă pe coloană:
 countryRegionId             0
year                        0
eia_primary_energy_44       0
eia_energy_intensity_47     0
eia_co2_emissions_4008      0
electricity_access          0
energy_use_per_capita       0
gdp_per_capita              0
renewables_share           12
dtype: int64
Duplicate finale (country, year): 0


In [16]:
final_data = final_data.sort_values(["countryRegionId", "year"])

final_data["renewables_share"] = (
    final_data
    .groupby("countryRegionId")["renewables_share"]
    .transform(lambda x: x.interpolate(method="linear"))
)

print(final_data.isna().sum())

countryRegionId            0
year                       0
eia_primary_energy_44      0
eia_energy_intensity_47    0
eia_co2_emissions_4008     0
electricity_access         0
energy_use_per_capita      0
gdp_per_capita             0
renewables_share           0
dtype: int64


## Dataset-ul final

Dataset-ul final rezultat conține:
- cheile: `countryRegionId`, `year`;
- indicatori energetici EIA;
- indicatori socio-economici WDI.

Acesta reprezintă un panel curat, fără duplicate și fără valori lipsă, pregătit
pentru analiza exploratorie a datelor și pentru etapele de feature engineering
și analiză statistică.

In [17]:
final_data.to_csv(
    "03_cleaning_aggregation_student3.csv",
    index=False
)